In [1]:
library(Seurat)
library(dplyr)
library(rdist)


Attaching SeuratObject


Attache Paket: ‘dplyr’


Die folgenden Objekte sind maskiert von ‘package:stats’:

    filter, lag


Die folgenden Objekte sind maskiert von ‘package:base’:

    intersect, setdiff, setequal, union




# Generate test data

In [2]:
# !wget https://zenodo.org/record/7041849/files/DatlingerBock2021.h5ad
sce_object <- zellkonverter::readH5AD('/Users/stefanpeidli/work/projects/scPerturb/package/notebooks/DatlingerBock2021.h5ad', X_name='counts')
seurat_object <- as.Seurat(sce_object, counts = 'counts', data = 'counts')
seurat_object

Registered S3 method overwritten by 'zellkonverter':
  method                from      
  py_to_r.numpy.ndarray reticulate



An object of class Seurat 
25904 features across 39194 samples within 1 assay 
Active assay: originalexp (25904 features, 0 variable features)

In [3]:
# # basic qc and pp
# sc.pp.filter_cells(adata, min_counts=1000)
# sc.pp.filter_genes(adata, min_cells=50)
# sc.pp.normalize_per_cell(adata)
# sc.pp.log1p(adata)
X = Seurat::GetAssayData(object = seurat_object, slot = "counts")
seurat_object[['ncounts']] <- Matrix::colSums(x = X)
seurat_object <- seurat_object[, seurat_object[['ncounts']]>1000]
seurat_object <- NormalizeData(seurat_object, normalization.method = "LogNormalize", scale.factor = 10000)
ncells <- Matrix::rowSums(x = X>0)
seurat_object <- seurat_object[ncells>50,]

# # high class imbalance
# adata = equal_subsampling(adata, 'perturbation', N_min=50)
# sc.pp.filter_genes(adata, min_cells=3)  # sanity cleaning
sdf <- seurat_object[[]] %>% as.data.frame()
sdf['barcode'] <- rownames(sdf)
sdf <- sdf %>% group_by(perturbation) %>% slice_sample(n=200)
seurat_object <- seurat_object[,colnames(seurat_object) %in% sdf$barcode]
X = Seurat::GetAssayData(object = seurat_object, slot = "counts")
ncells <- Matrix::rowSums(x = X>0)
seurat_object <- seurat_object[ncells>3,]

# # select HVGs
# n_var_max = 2000  # max total features to select
# sc.pp.highly_variable_genes(adata, n_top_genes=n_var_max, subset=False, flavor='seurat_v3', layer='counts')
seurat_object <- FindVariableFeatures(seurat_object, selection.method = "vst", nfeatures = 2000)

# sc.pp.pca(adata, use_highly_variable=True)
seurat_object <- ScaleData(seurat_object, features = rownames(seurat_object))
seurat_object <- RunPCA(seurat_object, features = VariableFeatures(object = seurat_object), verbose = FALSE)

Centering and scaling data matrix



In [4]:
# Subset to HVGs to save space
seurat_object <- seurat_object[VariableFeatures(object = seurat_object),]

In [7]:
# Save to data folder
path <- '../data/'

pca <- Embeddings(seurat_object, reduction = "pca")
metadata <- seurat_object[[]]

# GEX
library(Matrix)
mat = GetAssayData(object = seurat_object, slot = "counts")

print('Writing...')
writeMM(mat, paste0(path, 'matrix.mtx'))  # then gzip it
write.csv(rownames(seurat_object), paste0(path, 'genes.csv'))
write.csv(colnames(seurat_object), paste0(path, 'barcodes.csv'))
write.csv(pca, paste0(path, "/seurat_pca.csv"))
write.csv(metadata, paste0(path, "/seurat_metadata.csv"))



[1] "Writing..."


NULL

In [40]:
# read back in
expression_matrix <- Seurat::ReadMtx(
  paste0(path, 'matrix.mtx'),
  paste0(path, 'barcodes.csv'),
  paste0(path, 'genes.csv'),
  cell.column = 1,
  feature.column = 2,
  cell.sep = ",",
  feature.sep = ",",
  skip.cell = 1,
  skip.feature = 1,
)

latent <- read.csv(paste0(path, "/seurat_pca.csv"), row.names = 1)
metadata <- read.csv(paste0(path, "/seurat_metadata.csv"), row.names = 1)
colnames(expression_matrix) <- rownames(metadata)
seurat_object_reloaded <- CreateSeuratObject(counts = expression_matrix, meta.data = metadata)
seurat_object_reloaded[["pca"]] <- CreateDimReducObject(embeddings = as.matrix(latent), key = "pca_", assay = DefaultAssay(seurat_object_reloaded))


# E-test

In [133]:
#install.packages('energy')
#install.packages('Seurat')

#' Performs E-testing on a Seurat object
#'
#' Computes E-test statistics for each group in a Seurat object, using the E-distance in space given by reduction to the group defined by control.
#' @param seurat_object An object of class Seurat.
#' @param groupy An object of class character. Points to the column in the Seurat object's meta data that contains the group labels.
#' @param control An object of class character. The group that is used as the control.
#' @param reduction An object of class character. The reduction / embedding in seurat_object that is used to compute the E-distance in.
#' @return Returns an object of class data.frame. For each group contains the E-test p-value and the E-distance to control group.
#' @examples
#' # Add some code illustrating how to use the function
etest <- function(seurat_object, groupby = 'perturbation', control = 'control', reduction = 'pca', verbose = TRUE, permutations = 1000) {
    if (class(seurat_object)!='Seurat'){stop("The first argument must be a Seurat object.")}
    if (!(reduction %in% names(seurat_object@reductions))){
        if (reduction == 'pca') {
            if (verbose) {print('No PCA found in the Seurat object. Computing PCA now.')}
            seurat_object <- Seurat::RunPCA(seurat_object, features = VariableFeatures(object = seurat_object), verbose = FALSE)
        } else { stop("The specified reduction was not found in the Seurat object.")}
    }
    labels <- seurat_object[[]][[groupby]]
    groups <- unique(labels)
    if (!(control %in% groups)){stop("The specified control group was not found in the groupby column in the seurat_object metadata.")}
    emb <- Seurat::Embeddings(seurat_object, reduction = reduction)

    df <- data.frame(row.names = groups, pval = rep(NA, length(groups)))
    if (verbose) {print('Computing E-test statistics for each group.')}
    for (group in groups) {
        x <- as.matrix(emb)[labels==control,]
        y <- as.matrix(emb)[labels==group,]
        X <- rbind(x, y)
        d <- stats::dist(X)
        res <- energy::eqdist.etest(d, sizes=c(nrow(x), nrow(y)), distance=TRUE, R = permutations)
        df[group, 'pval'] <- res$'p.value'
        df[group, 'edist'] <- res$statistic
    }
    return(df)
}

In [134]:
df <- etest(seurat_object = seurat_object, groupby = 'perturbation', control = 'control', reduction = 'pca')

[1] "Computing E-test statistics for each group."


In [135]:
head(df)

,pval,edist
,<dbl>,<dbl>
ZAP70_1,0.108891109,22.68171
ZAP70_2,0.009990010,24.83029
LCK_1,0.006993007,31.65210
LCK_2,0.192807193,21.11392
LAT_1,0.001998002,27.59802
LAT_2,0.108891109,21.84110


# E-distance

In [160]:
#' Computes pairwise E-distances on a Seurat object
#'
#' Computes E-distance between all groups in a Seurat object in space given by reduction.
#' @param seurat_object An object of class Seurat.
#' @param groupy An object of class character. Points to the column in the Seurat object's meta data that contains the group labels.
#' @param reduction An object of class character. The reduction / embedding in seurat_object that is used to compute the E-distance in.
#' @param sample_correction An object of class logical. If TRUE, the E-distances are corrected for sample size. Will make it not a proper distance, leads to negative values.
#' @return Returns an object of class data.frame. For each group contains the E-test p-value and the E-distance to control group.
#' @examples
#' # Add some code illustrating how to use the function
edist <- function(seurat_object, groupby = 'perturbation', reduction = 'pca', sample_correction = FALSE, verbose = TRUE) {
    if (class(seurat_object)!='Seurat'){stop("The first argument must be a Seurat object.")}
    if (!(reduction %in% names(seurat_object@reductions))){
        if (reduction == 'pca') {
            if (verbose) {print('No PCA found in the Seurat object. Computing PCA now.')}
            seurat_object <- Seurat::RunPCA(seurat_object, features = VariableFeatures(object = seurat_object), verbose = FALSE)
        } else { stop("The specified reduction was not found in the Seurat object.")}
    }
    labels <- seurat_object[[]][[groupby]]
    groups <- unique(labels)
    emb <- Seurat::Embeddings(seurat_object, reduction = reduction)

    df <- setNames(data.frame(matrix(ncol = length(groups), nrow = length(groups)), row.names=groups), groups)
    if (verbose) {print('Computing E-test statistics for each group.')}
    completed_groups <- c()
    for (groupx in groups) {
        for (groupy in groups){
            if (groupy %in% completed_groups) {next}  # skip if already computed
            x <- as.matrix(emb)[labels==groupx,]
            y <- as.matrix(emb)[labels==groupy,]
            #res <- energy::edist(c(x,y), s=c(50,50), distance = FALSE)  # this is the original edist function by Rizzo

            N <- nrow(x)
            M <- nrow(y)

            dist_xy <- rdist::cdist(x,y)
            dist_x <- rdist::pdist(x)
            dist_y <- rdist::pdist(y)

            if (sample_correction) {
                ed <- 2 * (sum(dist_xy) / (N * M)) - (sum(dist_x) / (N * (N - 1))) - (sum(dist_y) / (M * (M - 1)))
            } else {
                ed <- 2 * (sum(dist_xy) / (N * M)) - (sum(dist_x) / (N * N)) - (sum(dist_y) / (M * M))
            }

            df[groupx, groupy] <- df[groupy, groupx] <- ed  # distance matrix is symmetric
        }
        completed_groups <- c(completed_groups, groupx)
    }
    return(df)
}

In [161]:
df <- edist(seurat_object, groupby = 'perturbation', reduction = 'pca', sample_correction = FALSE, verbose = TRUE)

[1] "Computing E-test statistics for each group."
